# import librarys

In [54]:

import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error , f1_score ,accuracy_score
from math import sqrt
from sklearn.model_selection import GridSearchCV
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_val_score
import joblib

# load data

In [55]:
df = pd.read_csv('data/customer_data.csv',names = ['CustomerId','CustomerCode','CustomerName','AreaName','IndustryName','OrderNumber','OrderDate','ProductCode','ProductName','Length'],header=None)

In [56]:
df.head()

,CustomerId,CustomerCode,CustomerName,AreaName,IndustryName,OrderNumber,OrderDate,ProductCode,ProductName,Length
0,1662,GEORGR,GEORGE GRANT ENGINEERING LTD,AUCKLAND SOUTH,STEEL FABRICATORS,1086081,2013-12-16 15:48:39.167,PM010+24004500,PLATE MEDIUM TENSILE GR350,1.0
1,1662,GEORGR,GEORGE GRANT ENGINEERING LTD,AUCKLAND SOUTH,STEEL FABRICATORS,1093839,2014-01-29 14:09:07.887,PM010+24004500,PLATE MEDIUM TENSILE GR350,1.0
2,1662,GEORGR,GEORGE GRANT ENGINEERING LTD,AUCKLAND SOUTH,STEEL FABRICATORS,(NUMBER),2014-06-04 00:00:00.000,PM010+24004500,PLATE MEDIUM TENSILE GR350,1.0
3,1662,GEORGR,GEORGE GRANT ENGINEERING LTD,AUCKLAND SOUTH,STEEL FABRICATORS,1132027,2014-06-05 17:53:39.143,PM010+24004500,PLATE MEDIUM TENSILE GR350,1.0
4,1662,GEORGR,GEORGE GRANT ENGINEERING LTD,AUCKLAND SOUTH,STEEL FABRICATORS,1147064,2014-07-22 11:25:19.037,PM010+24004500,PLATE MEDIUM TENSILE GR350,1.0


In [57]:
df['PurchaseCount'] = 1
pivoted_df = df.groupby(['CustomerId', 'ProductName']).sum().reset_index()

![this](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*b4M7o7W8bfRRxdMxtFoVBQ.png)
https://towardsdatascience.com/recsys-series-part-4-the-7-variants-of-matrix-factorization-for-collaborative-filtering-368754e4fab5

In [58]:

# create pivot table
pivoted_df = pivoted_df.pivot_table(index='CustomerId', columns='ProductName', values='PurchaseCount', fill_value=0)

In [59]:
pivoted_df 

ProductName,BLK PIPE LGT P/E(33.7OD) AS1074 C250,BRIGHT FREE CUTTING,COLORSTEEL CLOUD ENDURA G300,COLORSTEEL DESERT SAND ENDURA G300,COLORSTEEL ENDURA CLOUD ENDURA G300,COLORSTEEL ENDURA DESERT SAND (SF) G300,COLORSTEEL ENDURA DESERT SAND ENDURA (SF) G300,E.W.S.SQUARE TUBE,GALV PIPE MED S/S (114.1OD) AS1074 C250,PLATE AS 3678-450,PLATE MEDIUM TENSILE GR350,Plate Medium Tensile,RHS C350 PAINTED,RHS NOP AS 1163 C350 LO,RYDAL FLAT S1010,SHEET ELECTRO GALV,SHS C350 Painted,SHS NOP AS/NZS 1163 G450 LO,UNEQUAL ANGLE AS/NZS 3679 GRADE 300,ZINC ALUMINIUM G300 AZ150
CustomerId,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0
11,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,8,0
13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12279,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
12379,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12384,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [60]:
pivoted_df.iloc[0,1]

0

In [61]:
data = pivoted_df.copy()
for i in range(len(data)):
    total = data.iloc[i].sum().sum()
    for j in range(len(pivoted_df.iloc[0])):
        data.iloc[i,j] = round((data.iloc[i,j] / total) * 10 , 2)

In [62]:
data

ProductName,BLK PIPE LGT P/E(33.7OD) AS1074 C250,BRIGHT FREE CUTTING,COLORSTEEL CLOUD ENDURA G300,COLORSTEEL DESERT SAND ENDURA G300,COLORSTEEL ENDURA CLOUD ENDURA G300,COLORSTEEL ENDURA DESERT SAND (SF) G300,COLORSTEEL ENDURA DESERT SAND ENDURA (SF) G300,E.W.S.SQUARE TUBE,GALV PIPE MED S/S (114.1OD) AS1074 C250,PLATE AS 3678-450,PLATE MEDIUM TENSILE GR350,Plate Medium Tensile,RHS C350 PAINTED,RHS NOP AS 1163 C350 LO,RYDAL FLAT S1010,SHEET ELECTRO GALV,SHS C350 Painted,SHS NOP AS/NZS 1163 G450 LO,UNEQUAL ANGLE AS/NZS 3679 GRADE 300,ZINC ALUMINIUM G300 AZ150
CustomerId,,,,,,,,,,,,,,,,,,,,
2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,2.50,0.0,0.0,2.5,0.0,2.50,0.0,0.00,0.00,0.0
3,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,10.00,0.0,0.00,0.00,0.0
4,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,10.0,0.00,0.0,0.00,0.00,0.0
11,2.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.71,0.0,0.0,0.0,0.0,0.71,0.0,0.71,5.71,0.0
13,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,10.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12279,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,10.00,0.0,0.00,0.00,0.0
12379,10.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0
12384,10.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0


In [63]:
data.to_csv('123.csv')

In [64]:
data.iloc[0].unique()

array([0. , 2.5])

In [65]:
X = pivoted_df.values.T

# knn model

In [66]:

knn_model = NearestNeighbors(n_neighbors=10, metric='cosine')
knn_model.fit(pivoted_df)


NearestNeighbors(metric='cosine', n_neighbors=10)

In [67]:
def get_recommendations(customer_id):
    customer_pro = df[df['CustomerId'] == customer_id]['ProductName'].unique()
    customer_idx = pivoted_df.index.get_loc(customer_id)

    distances, indices = knn_model.kneighbors(pivoted_df.iloc[customer_idx, :].values.reshape(1, -1))
    
    # Get the product purchase counts for the 10 nearest neighbors
    neighbor_purchases = pivoted_df.iloc[indices[0], :]
    
    # Calculate the average purchase counts for each product among the 10 nearest neighbors
    avg_purchases = neighbor_purchases.mean(axis=0)
    
    # Sort the products by the average purchase counts and return the top 10
    top_products = avg_purchases.sort_values(ascending=False)[:10]
    # print(list(top_products.index))
    filter_products = [i for i in list(top_products.index) if i not in customer_pro]
    # Return the top 10 product codes
    return filter_products


In [68]:
customer_id = 1662
recommended_products = get_recommendations(customer_id)
print(recommended_products)


['UNEQUAL ANGLE AS/NZS 3679 GRADE 300', 'BRIGHT FREE CUTTING', 'SHS C350 Painted', 'SHEET ELECTRO GALV', 'RYDAL FLAT S1010', 'RHS NOP AS 1163 C350 LO', 'RHS C350 PAINTED', 'Plate Medium Tensile']


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


# Non-negative Matrix Factorization model

In [69]:
customer_id_to_idx = {cust_id: i for i, cust_id in enumerate(pivoted_df.index)}
customer_id_to_idx

{2: 0,
 3: 1,
 4: 2,
 11: 3,
 13: 4,
 16: 5,
 24: 6,
 27: 7,
 34: 8,
 36: 9,
 38: 10,
 42: 11,
 45: 12,
 48: 13,
 49: 14,
 53: 15,
 56: 16,
 59: 17,
 62: 18,
 63: 19,
 64: 20,
 67: 21,
 70: 22,
 80: 23,
 81: 24,
 82: 25,
 84: 26,
 85: 27,
 87: 28,
 88: 29,
 94: 30,
 97: 31,
 98: 32,
 103: 33,
 104: 34,
 111: 35,
 116: 36,
 117: 37,
 118: 38,
 120: 39,
 129: 40,
 132: 41,
 133: 42,
 135: 43,
 139: 44,
 142: 45,
 149: 46,
 154: 47,
 156: 48,
 160: 49,
 161: 50,
 164: 51,
 169: 52,
 171: 53,
 172: 54,
 174: 55,
 175: 56,
 176: 57,
 177: 58,
 181: 59,
 183: 60,
 184: 61,
 187: 62,
 192: 63,
 195: 64,
 200: 65,
 202: 66,
 212: 67,
 216: 68,
 219: 69,
 221: 70,
 224: 71,
 225: 72,
 237: 73,
 244: 74,
 248: 75,
 251: 76,
 255: 77,
 258: 78,
 260: 79,
 262: 80,
 268: 81,
 270: 82,
 273: 83,
 276: 84,
 277: 85,
 281: 86,
 282: 87,
 283: 88,
 284: 89,
 285: 90,
 287: 91,
 291: 92,
 292: 93,
 293: 94,
 295: 95,
 298: 96,
 302: 97,
 305: 98,
 307: 99,
 310: 100,
 313: 101,
 316: 102,
 320: 103,
 3

In [70]:

product_codes = data.columns.tolist()
purchase_matrix = data.to_numpy()
purchase_matrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [10.,  0.,  0., ...,  0.,  0.,  0.],
       [10.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [71]:

nmf = NMF(n_components=50, max_iter=1000)
W = nmf.fit_transform(purchase_matrix)
H = nmf.components_

# Get the latent factor matrix for customers
customer_latent_factors = W

# Get the latent factor matrix for products
product_latent_factors = H.T

# Calculate the reconstruction error
reconstruction_error = nmf.reconstruction_err_
print("Reconstruction error:", reconstruction_error)

# Calculate the RMSE and MSE of the reconstructed purchase matrix
pred_purchase_matrix = np.dot(W, H)
mse = mean_squared_error(purchase_matrix, pred_purchase_matrix)
rmse = np.sqrt(mse)
print("MSE:", mse)
print("RMSE:", rmse)


Reconstruction error: 2.2170012764855
MSE: 0.00010556474785090925
RMSE: 0.010274470684707278


In [72]:

# param_grid = {'n_components': [10, 20, 30, 40, 50], 
#               'alpha_W': [0.01, 0.1, 1, 10],
#               'alpha_H': [0.01, 0.1, 1, 10]}

# nmf = NMF()

# def nmf_mse(model, X):
#     W = model.fit_transform(X)
#     H = model.components_
#     return -mean_squared_error(X, W.dot(H))

# grid_search = GridSearchCV(nmf, param_grid, cv=5, n_jobs=-1, verbose=1, scoring=nmf_mse)
# grid_search.fit(data)

# print('Best hyperparameters:', grid_search.best_params_)
# print('Best score:', grid_search.best_score_)


In [73]:
joblib.dump(nmf,'models/nmf.pkl')

['models/nmf.pkl']

In [74]:

def recommend_products_nmf(customer_id, num_recs=10):
    # already customer buy products
    customer_pro = df[df['CustomerId'] == customer_id]['ProductName'].unique()
    customer_idx = customer_id_to_idx[customer_id]
    customer_vec = customer_latent_factors[customer_idx]
    rec_scores = product_latent_factors.dot(customer_vec)
    rec_indices = (-rec_scores).argsort()[:num_recs]
    return [product_codes[i] for i in rec_indices if product_codes[i] not in customer_pro][:5]


In [75]:
customer_id = 3
recommended_products = recommend_products_nmf(customer_id)
print(recommended_products)



['PLATE MEDIUM TENSILE GR350', 'RYDAL FLAT S1010', 'BLK PIPE LGT P/E(33.7OD) AS1074 C250', 'GALV PIPE MED S/S (114.1OD) AS1074 C250 ', 'RHS C350 PAINTED']


In [76]:
df[df['ProductName'] == 'BLK PIPE LGT P/E(33.7OD) AS1074 C250']

,CustomerId,CustomerCode,CustomerName,AreaName,IndustryName,OrderNumber,OrderDate,ProductCode,ProductName,Length,PurchaseCount
34,573,BRUNEN,BRUNTON ENGINEERING LTD,PALMERSTON NORTH,MANUFACTURERS,000000,2010-11-04 00:00:00.000,PB025L+0650,BLK PIPE LGT P/E(33.7OD) AS1074 C250,6.5,1
35,2864,NEWPSS,NEW PLYMOUTH STEEL SUPPLIES LTD,TARANAKI,RESELLER/COMPETITOR,935916,2012-03-09 00:00:00.000,PB025L+0650,BLK PIPE LGT P/E(33.7OD) AS1074 C250,6.5,1
36,4617,XCALEN,XCALIBAR ENGINEERING,AUCKLAND SOUTH,PROJECT ENGINEERS,936706,2012-03-13 00:00:00.000,PB025L+0650,BLK PIPE LGT P/E(33.7OD) AS1074 C250,6.5,1
37,3983,TEPARI,TE PARI PRODUCTS,NORTH OTAGO,MANUFACTURERS,(NUMBER),2013-07-05 00:00:00.000,PB025L+0650,BLK PIPE LGT P/E(33.7OD) AS1074 C250,6.5,1
38,690,CASHCH,CASH SALE - CHRISTCHURCH,CHCH EAST,UNDEFINED,(NUMBER),2012-10-15 00:00:00.000,PB025L+0650,BLK PIPE LGT P/E(33.7OD) AS1074 C250,6.5,1
...,...,...,...,...,...,...,...,...,...,...,...
5797,10951,KEATRA,Kea Trailers NZ Ltd,HAMILTON,MANUFACTURERS,2699078,2023-02-01 07:50:26.740,PB025L+0650,BLK PIPE LGT P/E(33.7OD) AS1074 C250,6.5,1
5798,13963,COVERC,COVERCORP LTD - AKLD,AUCKLAND EAST,UNDEFINED,(NUMBER),2023-03-01 00:00:00.000,PB025L+0650,BLK PIPE LGT P/E(33.7OD) AS1074 C250,6.5,1
5799,4203,TOTSOL,TOTAL SOLUTIONS LTD,AUCKLAND NORTH,GENERAL ENGINEERING,2733654,2023-03-09 07:02:23.987,PB025L+0650,BLK PIPE LGT P/E(33.7OD) AS1074 C250,6.5,1
5800,4203,TOTSOL,TOTAL SOLUTIONS LTD,AUCKLAND NORTH,GENERAL ENGINEERING,2733726,2023-03-09 07:02:30.310,PB025L+0650,BLK PIPE LGT P/E(33.7OD) AS1074 C250,6.5,1


In [77]:
df[df['CustomerId'] == customer_id]['AreaName'].unique()

array(['AUCKLAND WEST'], dtype=object)

In [78]:
df[df['CustomerId'] == customer_id]['IndustryName'].unique()

array(['MANUFACTURERS'], dtype=object)

In [79]:
Productname = 'UNEQUAL ANGLE AS/NZS 3679 GRADE 300'

In [80]:
sorted(df[df['ProductName'] == Productname]['IndustryName'].unique())

['DAIRY INDUSTRY',
 'DOOR MANUFACTURER',
 'FENCING',
 'FOOD MANUFACTURERS',
 'GENERAL ENGINEERING',
 'MANUFACTURERS',
 'MARINE INDUSTRY',
 'PROJECT ENGINEERS',
 'RESELLER/COMPETITOR',
 'SHEETMETAL ENGINEERS',
 'STEEL FABRICATORS',
 'STRUCTURAL CONSTRUCTION',
 'TRAINING INSTITUTE',
 'TRANSPORT ENGINEERS',
 'UNDEFINED']

In [81]:
sorted(df[df['ProductName'] == Productname]['AreaName'].unique())

['AUCKLAND CENTRAL',
 'AUCKLAND EAST',
 'AUCKLAND NORTH',
 'AUCKLAND SOUTH',
 'AUCKLAND WEST',
 'CENTRAL OTAGO',
 'CH COIL PROCESSING',
 'CHCH EAST',
 'CHCH NORTH',
 'CHCH SOUTH',
 'CHCH WEST',
 'COROMANDEL',
 'DUNEDIN OTAGO',
 'GISBORNE',
 'HAMILTON',
 'HAWKES BAY',
 'KATIKATI TO PAEROA',
 'LEVIN',
 'MARLBOROUGH',
 'MATAMATA-TIRAU',
 'MID CANTERBURY',
 'MOUNT - TEPUKE',
 'NELSON & BAYS',
 'NORTH CANTERBURY',
 'NORTH OTAGO',
 'NORTHLAND',
 'PALMERSTON NORTH',
 'ROTORUA',
 'SOUTH CANTERBURY',
 'SOUTH OTAGO',
 'SOUTHLAND',
 'STAINLESS',
 'TARANAKI',
 'TAUPO-TURANGI',
 'TAURANGA',
 'TOKOROA-PUTARURU',
 'WAIKATO',
 'WAIRARAPA DISTRICT',
 'WANGANUI',
 'WELLINGTON',
 'WEST COAST',
 'WHAK-KAWER-OPOTIKI']

## customer_id = 1662

1. using similarity and vector <br>
- ['BLK PIPE LGT P/E(33.7OD) AS1074 C250', 'SHS NOP AS/NZS 1163 G450 LO', 'SHEET ELECTRO GALV', 'E.W.S.SQUARE TUBE', 'GALV PIPE MED S/S (114.1OD) AS1074 C250 ']

2. using nmf <br>
- ['UNEQUAL ANGLE AS/NZS 3679 GRADE 300', 'RYDAL FLAT S1010', 'SHEET ELECTRO GALV', 'E.W.S.SQUARE TUBE', 'COLORSTEEL ENDURA CLOUD ENDURA G300']

3. cosine_similarity and biases to model
- ['Plate Medium Tensile', 'COLORSTEEL ENDURA CLOUD ENDURA G300', 'BLK PIPE LGT P/E(33.7OD) AS1074 C250', 'UNEQUAL ANGLE AS/NZS 3679 GRADE 300', 'PLATE AS 3678-450']

## cosine_similarity

In [82]:
def recommend_products_nmf(customer_id, num_recs=10):
    customer_pro = df[df['CustomerId'] == customer_id]['ProductName'].unique()
    customer_idx = customer_id_to_idx[customer_id]
    customer_vec = customer_latent_factors[customer_idx]
    
    # Calculate cosine similarity between customer vector and product latent factors
    similarities = cosine_similarity([customer_vec], product_latent_factors)[0]
    
    rec_scores = product_latent_factors.dot(customer_vec)
    
    # Combine cosine similarity scores and recommendation scores
    combined_scores = rec_scores + similarities
    
    # Get recommendation indices based on combined scores
    rec_indices = (-combined_scores).argsort()[:num_recs]
    
    return [product_codes[i] for i in rec_indices if product_codes[i] not in customer_pro][:5]

customer_id = 1662
recommended_products = recommend_products_nmf(customer_id)
print(recommended_products)

['Plate Medium Tensile', 'COLORSTEEL ENDURA CLOUD ENDURA G300', 'SHEET ELECTRO GALV', 'RYDAL FLAT S1010', 'RHS C350 PAINTED']


## biases to the factorization model

In [83]:
# Add biases to the factorization model
customer_bias = np.mean(purchase_matrix - np.dot(customer_latent_factors, product_latent_factors.T))
product_bias = np.mean(purchase_matrix - np.dot(customer_latent_factors, product_latent_factors.T))

# Predict the purchase matrix with biases
pred_purchase_matrix_with_biases = np.dot(customer_latent_factors, product_latent_factors.T) + customer_bias + product_bias

# Calculate the RMSE and MSE of the purchase matrix with biases
mse_with_biases = mean_squared_error(purchase_matrix, pred_purchase_matrix_with_biases)
rmse_with_biases = np.sqrt(mse_with_biases)
print("MSE with biases:", mse_with_biases)
print("RMSE with biases:", rmse_with_biases)

def recommend_products_nmf(customer_id, num_recs=10):
    customer_pro = df[df['CustomerId'] == customer_id]['ProductName'].unique()
    customer_idx = customer_id_to_idx[customer_id]
    customer_vec = customer_latent_factors[customer_idx]
    rec_scores = np.dot(product_latent_factors, customer_vec) + product_bias
    rec_indices = (-rec_scores).argsort()[:num_recs]
    return [product_codes[i] for i in rec_indices if product_codes[i] not in customer_pro][:5]

customer_id = 1662
recommended_products = recommend_products_nmf(customer_id)
print(recommended_products)


MSE with biases: 0.00010556474785090872
RMSE with biases: 0.010274470684707252
['Plate Medium Tensile', 'COLORSTEEL ENDURA CLOUD ENDURA G300', 'SHEET ELECTRO GALV', 'BLK PIPE LGT P/E(33.7OD) AS1074 C250', 'SHS C350 Painted']
